In [170]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Reshape
import matplotlib.pyplot as plt

In [171]:
(x_train, y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

In [172]:
class Sampling(layers.Layer):
    """Uses (mean, log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5* z_log_var) * epsilon





class Encoder(Model):
    def __init__(self):
        super(Encoder, self).__init__()
        
    
    def call(self, inputs, latent_dim, summary = True):
        x = Flatten()(inputs)
        x = Dense(512, activation="relu")(x)
        
        mean = Dense(latent_dim, name="mean")(x)
        std_dev = Dense(latent_dim, name="log_var")(x)
        
        sampled = Sampling()([mean, std_dev])
        
        self.encoder = Model(inputs, sampled, name = "Encoder")
        if summary:
            print(self.encoder.summary())
        return self.encoder


class Decoder(Model):
    def __init__(self, ):
        super(Decoder, self).__init__()
        
    def call(self, latent_input, output_shape, summary = True):
        x = Dense(512, activation="relu")(latent_input)
        x = Dense(784, activation="relu")(x)
        output = Reshape(output_shape)(x)
        
        self.decoder = Model(latent_input, output, name="Decoder")
        
        if summary:
            self.decoder.summary()
        return self.decoder


In [178]:
class VAE(Model):
    def __init__(self, img_shape, latent_dim, summary = True):
        super(VAE, self).__init__()
        
        self.inputs = tf.keras.Input(shape= img_shape)
        self.sampled = encoder(self.inputs, latent_dim )
        self.output_layer = decoder(self.sampled, img_shape)
#         self.latent_inputs = tf.keras.Input(shape= (latent_dim,))        
    
    def call(self, get_encoder = False, get_decoder = False):
        self.vae = Model(self.inputs, self.output_layer, name = "VAE")
        return self.vae
#         if get encoder
        
    def encoder(self,  inputs, latent_dim, ):
        x = Flatten()(inputs)
        x = Dense(512, activation="relu")(x)
        
        mean = Dense(latent_dim, name="mean")(x)
        std_dev = Dense(latent_dim, name="log_var")(x)
        
        sampled = Sampling()([mean, std_dev])
        
#         self.encoder = Model(inputs, sampled, name = "Encoder")
#         if summary:
#             print(self.encoder.summary())
        return sampled
    
    def decoder(self,latent_input, output_shape ):
        x = Dense(512, activation="relu")(latent_input)
        x = Dense(784, activation="relu")(x)
        output = Reshape(output_shape)(x)
        
#         self.decoder = Model(latent_input, output, name="Decoder")
        
#         if summary:
#             self.decoder.summary()
        return output
        
        
        

In [181]:
latent_dim = 2
img_shape = (28, 28, 1)

vae = VAE(input_shape, latent_dim)()

ValueError: The first argument to `Layer.call` must always be passed.